In [47]:
#Set up
from bs4 import BeautifulSoup as bts
import pandas as pd
import requests
import math #for rounding numbers
from datetime import datetime, timedelta
#from dateutil.relativedelta import relativedelta
import csv
import time

In [48]:
#Function for parsing the URLs
def cookSoup(url): 
    result = requests.get(url, headers = {"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [49]:
# Function for pagination - creating a list of urls from a category
def pageCreation(soup, cat_url, country, cat):
    url_list = [cat_url]
    total_prod = soup.find("div",{"class":"plp-bar-info svelte-1uqvrhu"}).find("span", {"class":"svelte-1uqvrhu"}).text
    #No of pages (rounding up all numbers) = math.ceil(int(total_prod)/40)
    #Create list of urls within the cat
    total_page = math.ceil(int(total_prod)/40)
    for i in range(1, total_page):
            page = f'{cat_url}?from={40 * i}&size=40'
            url_list.append(page)
    print (f'{country}_{cat}: There are {total_prod} products ({total_page} pages) in the category')
    return url_list

In [50]:
# Function to get main data
def getDecathlonData(country_url, url_list, country, cat):
    prod_list = []
    for url in url_list:
        page_soup = cookSoup(url)
        prod = page_soup.find_all(class_="dpb-holder loaded svelte-wmr4s2")
        for product in prod:
            link = product.find("a").attrs["href"]
            prod_cat = page_soup.find("h1").text
            prod_title = product.find("h2",{"class":"svelte-wmr4s2"}).text
            brand_name = product.find("strong",{"class":"svelte-wmr4s2"}).text
            prod_url = f'{country_url}{link}'
            
            #if all sku contains only 7 characters: prod_id = product.find("a").attrs["href"].partition("?mc=")[2][0:7]
            #taking sku's even in case of more than 7 character id's:
            if link.partition("?mc=")[2].rpartition('&')[0] == '':
                prod_id = link.partition("?mc=")[2]
            else:
                prod_id = link.partition("?mc=")[2].rpartition('&')[0]

            #product prices in case of product being discounted or not
            if product.find("div", {"class":"prc__active-price svelte-1kkqb6z"}) == None:
                reg_price = product.find("span", {"class":"prc__previous"}).text.replace('\n','').replace('*','')
                act_price = product.find("div", {"class":"prc__active-price svelte-1kkqb6z prc__active-price--sale"}).text.replace('\n','').replace('*','')
            else:
                reg_price = product.find("div", {"class":"prc__active-price svelte-1kkqb6z"}).text.replace('\n','').replace('*','')
                act_price = None
            
            #product sticker (2 cases: present vs. abse)

            if product.find("div", {"class": "sticker svelte-15lojui"}) == None:
                prod_sticker = None
            else:
                prod_sticker = product.find("div", {"class": "sticker svelte-15lojui"}).text.replace('\n','').replace('*','').replace(' ','')

            prod_list.append({'title': prod_title,
                             'sku': prod_id,
                             'regular price': reg_price,
                             'actual price' : act_price,
                             'brand': brand_name,
                             'url' : prod_url,
                             'sticker' : prod_sticker,
                             'category' : prod_cat})
            
    print(f'{country}_{cat}: {len(prod_list)} products have been scraped!')
        
    return prod_list

In [51]:
# Function to save data into csv:
def saveDecathlonData(country, cat, prod_list):
    fetch_date=f'{datetime.date(datetime.today())}'
    filename = f'{country}_{cat}_{len(prod_list)}_{fetch_date}.csv'.replace(" ","_")

    with open(filename, "w", encoding='utf-8-sig', newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter = ",")
        writer.writerow(["title", "sku", "reg_price", "act_price", "brand", "sticker", "category", "url"])

        for item in prod_list:
            writer.writerow([item['title'], item['sku'], item['regular price'], item['actual price'], item['brand'], item['sticker'], item['category'], item['url']])
    
    return print(f'{country}_{cat}: Data has been saved in {filename}')

In [52]:
#DE - Woman sportwear
cat_url = 'https://www.decathlon.de/browse/c0-damen/c1-sportbekleidung-damen/_/N-qjh2ro'
country = 'DE'
country_url = "https://www.decathlon.de"
cat = 'Sportbekleidung Damen'

soup = cookSoup(cat_url)
url_list = pageCreation(soup, cat_url, country, cat)
prod_list = getDecathlonData(country_url, url_list, country, cat)
saveDecathlonData(country, cat, prod_list)


DE_Sportbekleidung Damen: There are 4226 products (106 pages) in the category
DE_Sportbekleidung Damen: 4226 products have been scraped!
DE_Sportbekleidung Damen: Data has been saved in DE_Sportbekleidung_Damen_4226_2022-04-07.csv


## To-do
1. DONE: Remove the space AND * in price, SPACE in Sticker
2. DONE: Saving URL as a list --> Scrape SKU in each URL OR: trying to extract SKU from the URL
3. DONE: Pagination
4. DONE: Saving as csv. with country_category as file name
5. Wrap chunks of codes up into functions
6. Auto translate category? https://medium.com/analytics-vidhya/how-to-translate-text-with-python-9d203139dcf5
7. Make a list of different categories and countries
8. Loop over cat & countries